In [1]:
import geopandas as gpd
import pandas as pd
from unidecode import unidecode

## Open Geojson

In [2]:
munis_gdf = gpd.read_file("data/Lietuvos_Respublikos_savivaldyb%C4%97s.geojson")

In [3]:
munis_gdf["muni_name"] = (
    munis_gdf["sav_pav"]
    .str.split(" ")
    .str[0:2]
    .str.join(" ")
    .apply(unidecode)
    .str.lower()
)

## Get Open Population data

In [4]:
population_df = pd.read_csv("data/data-table.csv")

In [5]:
population_df["muni_name"] = population_df["Administrative territory"].apply(unidecode)

In [6]:
formatted_population_df = population_df[population_df["Time"] == 2015][
    ["Value", "muni_name"]
].merge(
    population_df[population_df["Time"] == 2025][["Value", "muni_name"]],
    on="muni_name",
    how="left",
    suffixes=("_2015", "_2025"),
)

In [7]:
formatted_population_df = formatted_population_df.dropna()

In [8]:
formatted_population_df["difference"] = (
    formatted_population_df["Value_2025"] - formatted_population_df["Value_2015"]
).astype(int)
formatted_population_df["percent"] = (
    (formatted_population_df["difference"] / formatted_population_df["Value_2015"])
    * 100
).round(decimals=1)

In [9]:
formatted_population_df = formatted_population_df[
    ~formatted_population_df["muni_name"].str.contains(
        r"county|Region", case=False, na=False
    )
].reset_index(drop=True)

In [10]:
mapped_cols = {
    "akmenes r.": "Akmene d. mun.",
    "alytaus r.": "Alytus d. mun.",
    "alytaus m.": "Alytus t. mun.",
    "anyksciu r.": "Anyksciai d. mun.",
    "birstono sav.": "Birstonas mun.",
    "birzu r.": "Birzai d. mun.",
    "druskininku sav.": "Druskininkai mun.",
    "elektrenu sav.": "Elektrenai mun.",
    "ignalinos r.": "Ignalina d. mun.",
    "jonavos r.": "Jonava d. mun.",
    "joniskio r.": "Joniskis d. mun.",
    "jurbarko r.": "Jurbarkas d. mun.",
    "kaisiadoriu r.": "Kaisiadorys d. mun.",
    "kalvarijos sav.": "Kalvarija mun.",
    "kauno m.": "Kaunas c. mun.",
    "kauno r.": "Kaunas d. mun.",
    "kazlu rudos": "Kazlu Ruda mun.",
    "kedainiu r.": "Kedainiai d. mun.",
    "kelmes r.": "Kelme d. mun.",
    "klaipedos m.": "Klaipeda c. mun.",
    "klaipedos r.": "Klaipeda d. mun.",
    "kretingos r.": "Kretinga d. mun.",
    "kupiskio r.": "Kupiskis d. mun.",
    "lazdiju r.": "Lazdijai d. mun.",
    "marijampoles sav.": "Marijampole mun.",
    "mazeikiu r.": "Mazeikiai d. mun.",
    "moletu r.": "Moletai d. mun.",
    "neringos sav.": "Neringa mun.",
    "pagegiu sav.": "Pagegiai mun.",
    "pakruojo r.": "Pakruojis d. mun.",
    "palangos m.": "Palanga t. mun.",
    "panevezio m.": "Panevezys c. mun.",
    "panevezio r.": "Panevezys d. mun.",
    "pasvalio r.": "Pasvalys d. mun.",
    "plunges r.": "Plunge d. mun.",
    "prienu r.": "Prienai d. mun.",
    "radviliskio r.": "Radviliskis d. mun.",
    "raseiniu r.": "Raseiniai d. mun.",
    "rietavo sav.": "Rietavas mun.",
    "rokiskio r.": "Rokiskis d. mun.",
    "sakiu r.": "Sakiai d. mun.",
    "salcininku r.": "Salcininkai d. mun.",
    "siauliu m.": "Siauliai c. mun.",
    "siauliu r.": "Siauliai d. mun.",
    "silales r.": "Silale d. mun.",
    "silutes r.": "Silute d. mun.",
    "sirvintu r.": "Sirvintos d. mun.",
    "skuodo r.": "Skuodas d. mun.",
    "svencioniu r.": "Svencionys d. mun.",
    "taurages r.": "Taurage d. mun.",
    "telsiu r.": "Telsiai d. mun.",
    "traku r.": "Trakai d. mun.",
    "ukmerges r.": "Ukmerge d. mun.",
    "utenos r.": "Utena d. mun.",
    "varenos r.": "Varena d. mun.",
    "vilkaviskio r.": "Vilkaviskis d. mun.",
    "vilniaus m.": "Vilnius c. mun.",
    "vilniaus r.": "Vilnius d. mun.",
    "visagino sav.": "Visaginas mun.",
    "zarasu r.": "Zarasai d. mun.",
}

In [11]:
munis_gdf["muni_name"] = munis_gdf["muni_name"].map(mapped_cols)

## Merge data and export

In [12]:
lithuanian_pop_gdf = munis_gdf.merge(
    formatted_population_df, on="muni_name", how="left"
)

In [13]:
lithuanian_pop_gdf = lithuanian_pop_gdf.to_crs(3346)
lithuanian_pop_gdf.to_file("data/lithuanian_pop.gpkg")